# Mix'n'match Mismatch Generation

This notebook is used to genrate mismatches for [Mismatch Finder](https://www.wikidata.org/wiki/Wikidata:Mismatch_Finder) via a request to [Mix'n'match](https://meta.wikimedia.org/wiki/Mix%27n%27match) data stores. Data will be formatted for upload given the [directions for creating a mismatch file](https://github.com/wmde/wikidata-mismatch-finder/blob/main/docs/UserGuide.md#creating-a-mismatches-import-file).

In [ ]:
#!pip install jupyter-black
#!pip install tensorflow
#!pip install aiohttp

In [ ]:
# %load_ext jupyter_black

In [ ]:
import ast
import json
import sys
import urllib

import numpy as np
import pandas as pd

PATH_TO_UTILS = "../"  # change based on your directory structure
sys.path.append(PATH_TO_UTILS)

from utils import check_mf_formatting

## Get Data

In [ ]:
mnm_mismatch_request_url = (
    "https://mix-n-match.toolforge.org/api.php?query=all_issues&mode=time_mismatch"
)

In [ ]:
with urllib.request.urlopen(mnm_mismatch_request_url) as url:
    mnm_mismatch_data = json.load(url)

In [ ]:
print(f"{len(mnm_mismatch_data['data']):,}")

In [ ]:
mnm_mismatch_data["data"][:2]

In [ ]:
mnm_mismatch_data_expanded = []
for d in mnm_mismatch_data["data"]:
    d["source"] = f"https://mix-n-match.toolforge.org/#/entry/{d['entry_id']}"
    d.pop("issue_id", None)
    d["time_mismatch"]["pid"] = d["time_mismatch"].pop("prop")
    d["time_mismatch"]["qid"] = d["time_mismatch"].pop("q")
    d["item_id"] = d["time_mismatch"]["qid"]

    mnm_mismatch_data_expanded.append(d)

In [ ]:
mnm_mismatch_data_expanded[:10]

In [ ]:
mnm_mismatch_data_expanded = list(filter(lambda d: d["time_mismatch"]["wd_time"] != d["time_mismatch"]["mnm_time"], mnm_mismatch_data_expanded))
len(mnm_mismatch_data_expanded)

In [ ]:
import pandas as pd
from numpy import NAN
from tqdm import tqdm

In [ ]:
acc = []
for entry in tqdm(mnm_mismatch_data_expanded):
    data = entry["time_mismatch"]
    req = f'https://www.wikidata.org/w/rest.php/wikibase/v0/entities/items/{entry["item_id"]}?_fields=statements'
    try:
        with urllib.request.urlopen(req) as url:
            wd_props = json.load(url)["statements"]
    except urllib.request.HTTPError as e:
        # Fixed in newer version https://stackoverflow.com/questions/67723860/python-urllib-request-urlopen-http-error-308-permanent-redirect
        print("Skipped", req)
        print(e)
        continue
    
    if "P1220" not in wd_props:  # https://www.wikidata.org/wiki/Property:P1220
        # No mix'n'match id, meaning it is automatic, not manual match & should be skipped
        continue

    with urllib.request.urlopen(f'https://mix-n-match.toolforge.org/api.php?query=get_entry&entry={entry["entry_id"]}') as url:
        ext_url = json.load(url)["data"]["entries"][entry["entry_id"]]["ext_url"]
    
    acc.append({
        "item_id": entry["item_id"],
        "statement_guid": wd_props[data["pid"]][0]["id"],
        "property_id": data["pid"],
        "wikidata_value": data["wd_time"],
        "meta_wikidata_value": NAN,
        "external_value": data["mnm_time"],
        "external_url": ext_url,
        "type": "statement",
    })

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
import asyncio
import aiohttp
import ssl

async def fetch(session, entry):
    data = entry["time_mismatch"]
    req = f'https://www.wikidata.org/w/rest.php/wikibase/v0/entities/items/{entry["item_id"]}?_fields=statements'
    async with session.get(req, ssl=ssl.SSLContext()) as response:
        wd_props = (await response.json())["statements"]
        pbar.update(0.5)
        
        if "P1220" not in wd_props:  # https://www.wikidata.org/wiki/Property:P1220
            # No mix'n'match id, meaning it is automatic, not manual match & should be skipped
            pbar.update(0.5)
            return

        url = f'https://mix-n-match.toolforge.org/api.php?query=get_entry&entry={entry["entry_id"]}'
        async with session.get(url, ssl=ssl.SSLContext()) as response:
            ext_url = (await response.json())["data"]["entries"][entry["entry_id"]]["ext_url"]
            pbar.update(0.5)
        
            return {
                "item_id": entry["item_id"],
                "statement_guid": wd_props[data["pid"]][0]["id"],
                "property_id": data["pid"],
                "wikidata_value": data["wd_time"],
                "meta_wikidata_value": NAN,
                "external_value": data["mnm_time"],
                "external_url": ext_url,
                "type": "statement",
            }


async def fetch_all(urls, loop):
    async with aiohttp.ClientSession(loop=loop) as session:
        results = await asyncio.gather(*[fetch(session, url) for url in urls], return_exceptions=True)
        return results


if __name__ == '__main__':
    pbar = tqdm(total=len(mnm_mismatch_data_expanded))
    loop = asyncio.get_event_loop()
    acc = loop.run_until_complete(fetch_all(mnm_mismatch_data_expanded, loop))
    pbar.close()


In [ ]:
mismatchDF = pd.DataFrame(acc)
mismatchDF

In [ ]:
check_mf_formatting(mismatchDF)
len(mismatchDF)

In [ ]:
mismatchDF.to_csv("first-4327-mismatches.csv")